<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [1]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

In [2]:
# datos

# cargar datos
cookie_cats = pd.read_csv('data/cookie_cats.csv')

cookie_cats.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [3]:
cookie_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [10]:
# transformacion

# verificar valores nulos
cookie_cats.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [12]:
# convertir columnas "retention_1" y "retention_7" a enteros
cookie_cats['retention_1'] = cookie_cats['retention_1'].astype(int)
cookie_cats['retention_7'] = cookie_cats['retention_7'].astype(int)

cookie_cats.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [14]:
# dividir grupos en "control" (gate_30) y "tratamiento" (gate_40)
grupo_cont = cookie_cats[cookie_cats["version"] == "gate_30"]
grupo_trat = cookie_cats[cookie_cats["version"] == "gate_40"]

In [15]:
# muestreo

# media retencion para 1 dia
retention_1_cont = grupo_cont['retention_1'].mean()
retention_1_trat = grupo_trat['retention_1'].mean()

# media retencion para 7 dias
retention_7_cont = grupo_cont['retention_7'].mean()
retention_7_trat = grupo_trat['retention_7'].mean()

# medias de retencion
retention_1_cont, retention_1_trat, retention_7_cont, retention_7_trat

(0.4481879194630872,
 0.44228274967574577,
 0.19020134228187918,
 0.18200004396667327)

In [20]:
# testeo para retention_1

# umbral
alpha = 0.05

# calcular exitos y tamaños para cada grupo
exitos_cont = sum(grupo_cont['retention_1'])
obs_cont = len(grupo_cont['retention_1'])
exitos_trat = sum(grupo_trat['retention_1'])
obs_trat = len(grupo_trat['retention_1'])

# realizar prueba proporciones de Z-test
z_score, p_value = proportions_ztest([exitos_cont, exitos_trat], [obs_cont, obs_trat])

# resultados
print('Z-score:', z_score)
print('P-valor:', p_value)

# comprobar resultado estadisticamente significativo
if p_value < alpha:
    print('La diferencia entre grupos es estadisticamente significativa')
else:
    print('No se encontro diferencia estadisticamente significativa entre grupos')

Z-score: 1.7840862247974725
P-valor: 0.07440965529691913
No se encontro diferencia estadisticamente significativa entre grupos


In [21]:
# testeo para retention_7

# umbral o nivel de significancia
alpha = 0.05

# calcular exitos y tamaños para cada grupo
exitos_cont = sum(grupo_cont['retention_7'])
obs_cont = len(grupo_cont['retention_7'])
exitos_trat = sum(grupo_trat['retention_7'])
obs_trat = len(grupo_trat['retention_7'])

# realizar prueba de proporciones de Z-test
z_score, p_value = proportions_ztest([exitos_cont, exitos_trat], [obs_cont, obs_trat])

# resultados
print('Z-score:', z_score)
print('P-valor:', p_value)

# comprobar resultado estadisticamente significativo
if p_value < alpha:
    print('La diferencia entre los grupos es estadisticamente significativa')
else:
    print('No se encontro una diferencia estadisticamente significativa entre los grupos')


Z-score: 3.164358912748191
P-valor: 0.001554249975614329
La diferencia entre los grupos es estadisticamente significativa


In [ ]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de tu testeo.

'''La diferencia observada en las tasas de retención a 1 día
entre los grupos (control y tratamiento) sugiere que el cambio realizado en la versión del juego (gate_40)
no tiene un impacto en la retención a corto plazo.
'''

''' 
La diferencia observada en las tasas de retención a 7 días
entre los grupos (control y tratamiento) sugiere que el cambio
en la versión del juego podría tener un impacto en la retención
a más largo plazo.
'''